# resize images from yesterday's presentation

i made a notebook presentation with a lot original images in it.
the document is HUGE.
we're going to resize them.

1. we start by making a dataframe of the notebook so it is easy to rip through.
2. resize an image with PIL
3. apply that to all the attached pngs
4. create and save a new notebook 

In [1]:
    from pandas import *
    import pandas, io, json, base64, copy
    from toolz.curried.operator import *
    from toolz.curried import *
    from pathlib import Path
    compose = compose_left
    from PIL import Image

the original file is `20MB`

In [2]:
ls -lathr 2025-05-01-present.ipynb

-rw-r--r--  1 tonyfast  staff    20M May  2 22:31 2025-05-01-present.ipynb


we load a dataframe of the original notebook document

In [3]:
df = pipe(
    data := pipe(
        "2025-05-01-present.ipynb",
        Path,
        Path.read_text,
        json.loads
    ),
    get("cells"),
    Series,
    methodcaller("apply", Series),
    methodcaller("set_index", "id")
)

we target the `attachments` contained in the markdown cells that have images pasted in them

In [4]:
    attachments = (
        df.attachments
        .dropna()
        .apply(
            dict.items
        ).explode()
        .dropna()
        .apply(
            compose(list, partial(zip, "key value".split()), dict, Series)
        )
    )

extract the specfic png values in the document

In [5]:
    pngs = attachments.set_index("key").value.apply(get("image/png"))

a function to <dfn>resize</dfn> a png base64 encoded string

In [6]:
    def resize(object: "image/png"):
        img = Image.open(io.BytesIO(base64.b64decode(object)))
        w = 800
        if img.size[0] < w:
            return object
        h = int(img.size[1]/img.size[0] * w)
        img = img.resize((w,h))
        img.save(new := io.BytesIO(), "png")
        return base64.b64encode(new.getvalue()).decode()
    

apply the `resize` function to all the original pngs making smaller images

In [7]:
    new_pngs = dict(pngs.apply(resize))

create a new notebook to hold the smaller images

In [8]:
    new_data = copy.deepcopy(data)

insert the smaller images into the new document

In [9]:
for cell in new_data["cells"]:
    if attachments := cell.get("attachments"):
        for k, v in attachments.items():
            if k in new_pngs:
                attachments[k] = {"image/png": new_pngs[k]}

save the file

In [10]:
Path("2025-05-01-present-small.ipynb").write_text(json.dumps(new_data, indent=2))

11162684

the resulting file is half the size

In [11]:
ls -lathr 2025-05-01-present*.ipynb

-rw-r--r--  1 tonyfast  staff    20M May  2 22:31 2025-05-01-present.ipynb
-rw-r--r--  1 tonyfast  staff    11M May  2 23:13 2025-05-01-present-small.ipynb
